In [1]:
import math
from collections import deque

In [2]:
Point = tuple[float, float]

# Create a board
class Board:
    xp, yp = [0, -1, 0, 1], [-1, 0, 1, 0]
    EPSILON = 1e-7

    def __init__(self, vertices: tuple[Point]):
        self.vertices = vertices
        assert len(self.vertices) > 2

        self.size = 0
        for Point in vertices:
            self.size = max(self.size, Point[0], Point[1])
        self.size = math.ceil(self.size)

        self.board = [[False for _ in range(self.size)]
                      for _ in range(self.size)]
        self.__calculate_board()

    def __crossed(self, cx: int, cy: int, nx: int, ny: int) -> bool:
        def __counter_clockwise(a: Point, b: Point, c: Point) -> float:
            return (a[0]-b[0])*c[1] + (b[0]-c[0])*a[1] + (c[0]-a[0])*b[1]

        def __different_half_plane(x: Point, y: Point, a: Point, b: Point) -> bool:
            return __counter_clockwise(x, a, b) * __counter_clockwise(y, a, b) < self.EPSILON

        def __intersected(c: Point, n: Point, a: Point, b: Point) -> bool:
            return __different_half_plane(c, n, a, b) and __different_half_plane(a, b, c, n)

        cx, cy = cx + .5, cy + .5
        nx, ny = nx + .5, ny + .5
        for i in range(len(self.vertices)):
            a, b = self.vertices[i-1], self.vertices[i]
            if __intersected((cx, cy), (nx, ny), a, b):
                return True
        return False

    def __calculate_board(self):
        queue = deque()
        queue.append((0, 0))
        while len(queue):
            cx, cy = queue.popleft()
            for xa, ya in zip(self.xp, self.yp):
                nx, ny = cx + xa, cy + ya
                if self.__crossed(cx, cy, nx, ny):
                    continue
                if self.board[nx][ny]:
                    continue
                self.board[nx][ny] = 1
                queue.append((nx, ny))

    def __repr__(self):
        s = ""
        for y in range(self.size-1, -1, -1):
            for x in range(self.size):
                s += 'X' if self.board[x][y] else '-'
            s += '\n'
        return s

BOARD1 = (.0, .0), (100.0, .0), (100.0, 50.0), (.0, 50.0)
BOARD2 = (.0, .0), (.0, 386.6025), (150.0, 236.6025), (150.0, 86.6025)
BOARD3 = (.0, .0), (450.0, .0), (450.0, 150.0), (150.0, 150.0), (300.0, 600.0), (.0, 450.0)



In [ ]:
# Draw manually
with open("figure1.txt", "w") as file:
    print(Board(BOARD1), file=file)

with open("figure2.txt", "w") as file:
    print(Board(BOARD2), file=file)

with open("figure3.txt", "w") as file:
    print(Board(BOARD3), file=file)